<a href="https://colab.research.google.com/github/raghu-sanikommu/learn-genai/blob/main/nlq-sql/NLQ_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0: Installation and Imports

In [1]:
!pip install numpy pandas

In [2]:
import pandas as pd
import numpy as np
import sqlite3

# 1: Load sqlite from csv

In [40]:
conn = sqlite3.connect('students.db')

df = pd.read_csv('student-dataset.csv')
print(df.head(1))
df.to_sql('student', conn, if_exists='replace', index=False)

print("-----------------------")

df = pd.read_csv('college.csv')
print(df.head(1))
df.to_sql('college', conn, if_exists='replace', index=False)

conn.close()

   id       name nationality    city  latitude  longitude gender  \
0   0  Kiana Lor       China  Suzhou     31.31     120.62      F   

   ethnic.group  age  english.grade  math.grade  sciences.grade  \
0           NaN   22            3.5         3.7             3.1   

   language.grade  portfolio.rating  coverletter.rating  refletter.rating  
0             1.0                 4                 4.0                 4  
-----------------------
                       Name PredominantDegree HighestDegree FundingModel  \
0  Alabama A & M University        Bachelor's      Graduate       Public   

      Region      Geography  AdmissionRate  ACTMedian  SATAverage  \
0  Southeast  Mid-size City         0.8989         17         823   

   AverageCost  Expenditure  AverageFacultySalary  MedianDebt  \
0        18888         7459                  7079     19500.0   

   AverageAgeofEntry  MedianFamilyIncome  MedianEarnings  
0          20.629999             29039.0           27000  


# 2: Form Schema String of sqlite

In [41]:
# Connect to sqlite
conn = sqlite3.connect('students.db')
cursor = conn.cursor()

# Retrieve table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Form the schema string
schema_str = ""

# Get schema definitions
for table_name in tables:
  table_name = table_name[0]  # Extract table name from tuple
  cursor.execute(f"PRAGMA table_info({table_name});")
  schema = cursor.fetchall()

  # Get first row data
  cursor.execute(f"SELECT * FROM {table_name} LIMIT 1")
  first_row = list(cursor.fetchone())

  schema_str += f"Table: '{table_name}' \n"
  schema_str += "Columns: "
  schema_str += str([(column[1], column[2], first_row[index]) for index, column in enumerate(schema)]) # Each column is a tuple with (cid, name, type, notnull, dflt_value, pk)
  schema_str += "\n \n"


print(schema_str)

# Close the connection
conn.close()

Table: 'student' 
Columns: [('id', 'INTEGER', 0), ('name', 'TEXT', 'Kiana Lor'), ('nationality', 'TEXT', 'China'), ('city', 'TEXT', 'Suzhou'), ('latitude', 'REAL', 31.31), ('longitude', 'REAL', 120.62), ('gender', 'TEXT', 'F'), ('ethnic.group', 'REAL', None), ('age', 'INTEGER', 22), ('english.grade', 'REAL', 3.5), ('math.grade', 'REAL', 3.7), ('sciences.grade', 'REAL', 3.1), ('language.grade', 'REAL', 1.0), ('portfolio.rating', 'INTEGER', 4), ('coverletter.rating', 'REAL', 4.0), ('refletter.rating', 'INTEGER', 4)]
 
Table: 'college' 
Columns: [('Name', 'TEXT', 'Alabama A & M University'), ('PredominantDegree', 'TEXT', "Bachelor's"), ('HighestDegree', 'TEXT', 'Graduate'), ('FundingModel', 'TEXT', 'Public'), ('Region', 'TEXT', 'Southeast'), ('Geography', 'TEXT', 'Mid-size City'), ('AdmissionRate', 'REAL', 0.8989), ('ACTMedian', 'INTEGER', 17), ('SATAverage', 'INTEGER', 823), ('AverageCost', 'INTEGER', 18888), ('Expenditure', 'INTEGER', 7459), ('AverageFacultySalary', 'INTEGER', 7079), ('

# 3: Prompt for NLQ & LLM Connection

In [42]:
llm_prompt = lambda question: f"""
  You are an expert in converting English questions to SQL query!
  The SQL database has the following tables and their respective columns:

  # SQL Schema BEGIN
  {schema_str}
  # SQL Schema END

  Explanation of above schema:
  "Table" - Refers to the name of SQL Table
  "Columns" - Refers to the columns of Table. It's a list of tuples and each tuple contain
    1. Column Name, at index 0
    2. Column Type, at index 1
    3. Sample Data, at index 2

  Few Shot Examples:
  User Query: How many entries of student records are present?,
  Response: SELECT COUNT(*) FROM student;

  Important Note:
  The Response should be a simple string and do not wrap between ```sql ```.

  Now, given below is the user query. Convert it to SQL query:
  User Query: {question}
  Response:
"""

import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
llm_model = 'gemini-1.5-flash-latest'

genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel(model_name=llm_model)

# 4: NLQ in action

In [46]:
# question = "How many entries of records are present in student table?"
# question = "List all colleges with admission rate more than 80%"
question = "How many students are there from china?"

response = gemini_model.generate_content([llm_prompt(question)])
print(response.text)

SELECT COUNT(*) FROM student WHERE nationality = 'China' 



# 5: Final Output

In [47]:
conn = sqlite3.connect('students.db')

cursor = conn.cursor()

cursor.execute(response.text)
results = cursor.fetchall()

df = pd.DataFrame(results)
print(df)

conn.close()

    0
0  13
